In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set()

In [54]:
from datetime import datetime, date, time

dt = datetime.now()
dt

datetime.datetime(2023, 12, 15, 16, 1, 43, 18118)

In [65]:
import datetime

dt2 = datetime.datetime(year=3000, month=1, day=17)
delta = datetime.timedelta(days=479)
pep = dt2 - delta
pep.date()

datetime.date(2998, 9, 25)

In [136]:
datetime.datetime.strptime('2016-11-12', '%Y-%M-%d')

datetime.datetime(2016, 1, 12, 0, 11)

In [140]:
dt.replace(year=2000)

datetime.datetime(2000, 12, 15, 16, 12, 52, 406853)

In [123]:
dt = datetime.datetime.now()
dt.strftime("Now is the %H:%M:%S")

'Now is the 16:12:52'

In [49]:
def get_data(path):
    data = []
    with open(path, 'r') as f:
        for line in f.readlines():
            data.append(line.replace('\n',''))
    return data

data = get_data('data.txt')
data[2]

'Matteo,Foglai,Barbiere'

In [50]:
def cognome(name, data):
    data2 = []
    for i in data:
        data_list = i.split(',')
        if data_list[1].startswith(name):
            data2.append(data_list)
    return data2

cognome('Pi', data)

[['Peppa', 'Pig', 'Muratore'], ['George', 'Pig', 'Chef']]

In [53]:
def update(data, path):
    with open(path, 'a') as f:
        for i in data:
            f.write(str(i))

update(list(range(3)), 'data.txt')